In [1]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3

In [2]:
!pip install transformers

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments


In [4]:
import csv

csv_file_path = "/content/intent.csv"
encoding = "latin-1"

In [5]:
import chromadb
chroma_client = chromadb.Client()

In [6]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [7]:
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
sentence_transformer_ef = SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# Create a collection in ChromaDB and add documents with embeddings and metadata
collection = chroma_client.create_collection(name="intent_collection", embedding_function=sentence_transformer_ef)

In [9]:
# Create a collection in ChromaDB and add documents with embeddings and metadata
collection = chroma_client.delete_collection(name="intent_collection")

In [17]:
# Initialize lists to store documents, metadatas, and ids
documents = []
metadatas = []
ids = []

# Read data from the CSV file
with open(csv_file_path, 'r', encoding= encoding) as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row if exists
    for row in reader:
        pattern = row[0]  # Assuming the first column contains patterns
        intent = row[1]   # Assuming the second column contains intents

        # Add pattern to documents list
        documents.append(pattern)

        # Create metadata for the pattern
        metadata = {"intent": intent}
        metadatas.append(metadata)

        # Generate ID for the pattern
        pattern_id = f"{intent}_{len(documents)}"
        ids.append(pattern_id)


In [18]:
documents

['Need help',
 'I need some info',
 'Will you help me?',
 'What else can you do?',
 'What do you do?',
 'What can you help me with?',
 'What can you do?',
 'What are you good at?',
 'Need help plz',
 'How can you help me',
 'Can you help me?',
 'How can you assist?',
 'How can you help me?',
 'How do you help?',
 'How to help you?',
 'How will you assist me?',
 'i need help',
 'Can you tell some info',
 'Help me please',
 'Is this a bot?',
 'Is there a bot chatting to me?',
 'Is it automated message?',
 'Computer based pely',
 'Bot or human?',
 'Bot is chatting with me?',
 'Are you system generated message?',
 'Are you robot?',
 'Are you machine?',
 'Are you just computer?',
 'Are you a robot?',
 'Are you a person?',
 'Are you a machine?',
 'Are you a chatbot?',
 'Are you a bot?',
 'Are these automated messages?',
 'Am I talking to a bot?',
 'You are a boy are girl?',
 'You are a bot or human?',
 'This is a machine?',
 'This is a chatbot?',
 'System generated reply',
 'Sir you system g

In [19]:
metadatas

[{'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.assist'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'commonQ.bot'},
 {'intent': 'common

In [20]:
ids


['commonQ.assist_1',
 'commonQ.assist_2',
 'commonQ.assist_3',
 'commonQ.assist_4',
 'commonQ.assist_5',
 'commonQ.assist_6',
 'commonQ.assist_7',
 'commonQ.assist_8',
 'commonQ.assist_9',
 'commonQ.assist_10',
 'commonQ.assist_11',
 'commonQ.assist_12',
 'commonQ.assist_13',
 'commonQ.assist_14',
 'commonQ.assist_15',
 'commonQ.assist_16',
 'commonQ.assist_17',
 'commonQ.assist_18',
 'commonQ.assist_19',
 'commonQ.bot_20',
 'commonQ.bot_21',
 'commonQ.bot_22',
 'commonQ.bot_23',
 'commonQ.bot_24',
 'commonQ.bot_25',
 'commonQ.bot_26',
 'commonQ.bot_27',
 'commonQ.bot_28',
 'commonQ.bot_29',
 'commonQ.bot_30',
 'commonQ.bot_31',
 'commonQ.bot_32',
 'commonQ.bot_33',
 'commonQ.bot_34',
 'commonQ.bot_35',
 'commonQ.bot_36',
 'commonQ.bot_37',
 'commonQ.bot_38',
 'commonQ.bot_39',
 'commonQ.bot_40',
 'commonQ.bot_41',
 'commonQ.bot_42',
 'commonQ.bot_43',
 'commonQ.bot_44',
 'commonQ.bot_45',
 'commonQ.how_46',
 'commonQ.how_47',
 'commonQ.how_48',
 'commonQ.how_49',
 'commonQ.how_50',
 '

In [21]:
# Add documents with embeddings and metadata to the collection
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [22]:
# user le queries garepaxi calculate the embeddings for the query get top 10 results and based on that result ask "LLM" to determine the user intent

query_text = "too much time for approval"
query_embedding = sentence_transformer_ef([query_text])[0]

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=10,
    include=['documents']
)

In [23]:
retrieved_documents = results['documents']
for document in retrieved_documents:
    print(document)

['Too much time for approval', 'Approval time is too long.', 'Time for approval is very much', 'Approval time is very much', 'Time you need for approval', 'Wh are you taking so time time for approval?', 'Why is this approval taking so much time?', 'Approval is taking very much time...', 'What is the time duration for approval?', 'The loan approval is going for a long time']


Now use LLM to determine the user intent

In [28]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 language model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define function to generate text from GPT-2
def generate_text(prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Define function to retrieve top documents
def retrieve_top_documents(query_text):
    # Implement your document retrieval mechanism here
    # Return top documents based on similarity with query text
    top_documents = []  # Replace this with your retrieval mechanism
    return top_documents

# Retrieve top 10 documents and extract intents
top_documents = retrieve_top_documents(query_text)
extracted_intents = extract_intents(top_documents)  # Implement your intent extraction mechanism

# Aggregate intents
intent_counts = {}
for intent in extracted_intents:
    intent_counts[intent] = intent_counts.get(intent, 0) + 1

# Use Language Model for Intent Determination
aggregated_intent = max(intent_counts, key=intent_counts.get)

# Generate response using aggregated intent
response = generate_text(aggregated_intent)
print("User Intent:", aggregated_intent)
print("Response:", response)


ValueError: max() arg is an empty sequence